In [5]:
from keras.datasets import mnist

In [6]:
(X_train_0, Y_train_0), (X_test_0, Y_test_0) = mnist.load_data()

# 2. 데이터 준비 및 탐색

## 1) 데이터 확인하기

In [7]:
print("학습 데이터 수 : %d"%(X_train_0.shape[0]))
print("평가 데이터 수 : %d"%(X_test_0.shape[0]))

학습 데이터 수 : 60000
평가 데이터 수 : 10000


### - 입력 학습 데이터 :  X_train[0] 이미지 출력

In [ ]:
import matplotlib.pyplot as plt

In [4]:
plt.imshow(X_train_0[0], cmap='Greys')
plt.show()

NameError: name 'plt' is not defined

### - 출력 학습 데이터(레이블) : Y_train

In [ ]:
print("class : %d " % (Y_train_0[0]))

### - 입력 학습 데이터 : X_train[0] 벡터 출력

In [ ]:
import sys

In [ ]:
for x in X_train_0[0]:
    for i in x:
        sys.stdout.write('%4d' % i)
    sys.stdout.write('\n')

## 2) 데이터 정규화 :  {0 ~ 255} -> {0.0 ~ 1.0}

In [ ]:
from tensorflow.python.keras.utils import np_utils

In [ ]:
X_train = X_train_0.reshape(X_train_0.shape[0], 28, 28, 1).astype('float32') / 255
X_test = X_test_0.reshape(X_test_0.shape[0], 28, 28, 1).astype('float32') / 255

## 3) 출력값을 0~9 카테고리 벡터로 변형

In [ ]:
Y_train = np_utils.to_categorical(Y_train_0, 10)
Y_test = np_utils.to_categorical(Y_test_0, 10)

print(Y_train[0])  #작업 확인용 출력

# 3. 분석 모델 구축 : CNN 모델

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint,EarlyStopping

import os  #모델을 저장할 디렉토리 확인 및 만들기

In [ ]:
# 컨볼루션 신경망의 설정
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,  activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 최적화 모델 저장

In [ ]:
# 모델 최적화 설정
MODEL_DIR = '../14장_data/model/'
if not os.path.exists(MODEL_DIR): 
    os.mkdir(MODEL_DIR)

modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# 모델의 실행
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), \
                    epochs=30, batch_size=200, verbose=0, \
                    callbacks=[early_stopping_callback,checkpointer])

# 4. 모델 평가 및 시각화

## 1) 모델 정확도 

In [ ]:
# 모델 평가 (정확도) 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

## 2) 모델 평가 결과 시각화 : loss

In [ ]:
import numpy as np

In [ ]:
# 평가 데이터셋의 오차
y_vloss = history.history['val_loss']

# 학습 데이터셋의 오차
y_loss = history.history['loss']

In [ ]:
# 오차 그래프
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

# 5.  숫자 분류 

## 1) X_train[0]을 CNN 분류모델에 입력하여 분류 클래스 확인하기

In [ ]:
test_num = X_train[0].reshape(-1, 28, 28, 1)

In [ ]:
prediction = model.predict(test_num)

In [ ]:
prediction #작업 확인용 출력

In [ ]:
print('입력한 이미지는 %f 확률로 %d 입니다!'%(prediction[0, prediction.argmax()], prediction.argmax()))

## 2) 새로운 숫자 이미지 파일을 모델에 입력하여 분류 클래스 확인하기

In [ ]:
from PIL import Image

In [ ]:
test = Image.open('../14장_data/test7.jpg')
test.mode

In [ ]:
# 흑백 이미지으로 변환하기
test_gray = test.convert("L") 
test_gray.mode #작업 확인용 출력

In [ ]:
test_gray # 확인용 이미지 출력

In [ ]:
test_gray = test_gray.resize((28,28)) #분석 모델의 입력층 사이즈로 변환

In [ ]:
test_data = np.asarray(test_gray)

# 이미지 벡터 출력
for x in test_data:
    for i in x:
        sys.stdout.write('%4d' % i)
    sys.stdout.write('\n')

In [ ]:
plt.imshow(test_data, cmap='Greys')
plt.show()

In [ ]:
#벡터값 반전 시키기
test_data_v = []

for i in range(28):
    row = []
    for j in range(28):                
        row.append(255 - test_data[i][j])
    test_data_v.append(row)     

In [ ]:
# 이미지 벡터 출력-2
for x in test_data_v:
    for i in x:
        sys.stdout.write('%4d' % i)
    sys.stdout.write('\n')

In [ ]:
plt.imshow(test_data_v, cmap='Greys')
plt.show()

### - 이미지를 분석 모델의 입력층 형태에 맞게 변환하기

In [ ]:
test_data_v2 = np.asarray(test_data_v)
test_data_v2 = test_data_v2.reshape(-1, 28, 28, 1).astype('float32') / 255

test_data_v2.shape  #작업 확인용 출력 : (샘플개수, 가로픽셀수, 세로픽셀수, 색상수)

### - 모델 실행 및 결과 확인

In [ ]:
prediction_new = model.predict(test_data_v2)

prediction_new #작업 확인용 출력

In [ ]:
test_class = prediction_new.argmax()

print('입력한 이미지는 %f 확률로 %d 입니다!'%(prediction_new[0, test_class]*100, test_class))